In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val userSdk = UserSdk()
val username = "Johann-9.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=cfdaf5d6-260c-493d-9f9d-e91ac5ea0f0d, username=Johann-9.1)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import kotlinx.coroutines.runBlocking

val fundSdk = FundSdk()
val expenseFund = runBlocking {
    fundSdk.listFunds(user.id).items.find { it.name.value == "Expenses" }
}
expenseFund

FundTO(id=aae35afa-b3dc-4d5f-b5d3-d692b397c2bc, name=Expenses)

In [4]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val labels = labelsOf("basic", "home", "shopping_services", "transport", "fun", "gifts", "development")
    val request = CreateReportViewTO(reportViewName, expenseFund.id, ReportViewType.EXPENSE, Currency.RON, labels)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=764354fd-903d-419d-b72b-62a92ed29d28, name=Expense report, fundId=aae35afa-b3dc-4d5f-b5d3-d692b397c2bc, type=EXPENSE, currency=Currency(value=RON), labels=[basic, home, shopping_services, transport, fun, gifts, development])

In [5]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=764354fd-903d-419d-b72b-62a92ed29d28, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-12-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=2019-01-01, amount=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=2019-02-01, amount=-4301.32, value=ValueReportTO(start=2481.52, end=5185.08, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=2019-03-01, amount=-9171.45, value=ValueReportTO(start=5185.08, end=11301.26, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=2019-04-01, amount=-6967.76, value=ValueReportTO(start=11301.26, end=11525.51717828, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=2019-05-01, amount=-9043.57, value=ValueReportTO(start=11525.51717828, end=10647.13717828, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=2019-06-01, amount=-7024.09, value=ValueReportTO(start=10647.13717828, end=11878.93717828, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=2019-07-01

In [6]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val amount: Double)

val monthlyReportDF = report.data
    .map {
        MonthlyExpense(it.timeBucket, it.amount.negate().toDouble())
    }
    .toDataFrame()

plot(monthlyReportDF) {
    y("amount")
    bars {
        x("month") {

        }
        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rQN3Ah"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"amount":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,0.0,0.0],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"amount",
"x":"month"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("rQN3Ah");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 Jun 
 
 
 
 
 
 
 
 
 Jul 
 
 
 
 
 
 
 
 
 Aug 
 
 
 
 
 
 
 
 
 Sep 
 
 
 
 
 
 
 
 
 Oct 
 
 
 
 
 
 
 
 
 Nov 
 
 
 
 
 
 
 
 
 Dec 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 8,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 12,000 
 
 
 
 
 
 
 14,000 
 
 
 
 
 
 
 16,000 
 
 
 
 
 
 
 18,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 22,000 
 
 
 
 
 
 
 24,000 
 
 
 
 
 
 
 26,000 
 
 
 
 
 
 
 
 
 Monthly expenses 
 
 
 
 
 amount 
 
 
 
 
 month

In [7]:
val df = dataFrameOf(
    "month" to report.data.map { it.timeBucket }.let { it + it},
    "value" to report.data.map { it.value.start } + report.data.map { it.value.end },
    "type" to List(report.data.size) { "startValue" } + List(report.data.size) { "endValue" }
)

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Pr8p9h"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Expense fund value"
},
"mapping":{
},
"data":{
"&merged_groups":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12],
"type":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"value":[0.0,2481.52,5185.08,11301.26,11525.51717828,10647.13717828,11878.93717828,8646.18717828,-6541.22091372,-873.90868196,4053.53131804,4053.53131804,2481.52,5185.08,11301.26,11525.51717828,10647.13717828,11878.93717828,8646.18717828,-6541.22091372,-873.90868196,4053.53131804,4053.53131804,4053.53131804]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#fc8452","#5470c6"],
"limits":["startValue","endValue"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("Pr8p9h");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M45.27221042399469 329.12418088493115 L45.27221042399469 329.12418088493115 L129.31044534278044 265.31701358372504 L205.2159478500762 195.80054576204773 L289.25418276886194 38.53559504462828 L370.5815068838201 32.769284358835534 L454.61974180260586 55.355013917345445 L535.9470659175568 23.68181818181813 L619.9853008363498 106.8053151082882 L704.0235357551355 497.3181818181818 L785.3508598700864 351.59493972459325 L869.3890947888794 224.895985677616 L950.7164189038303 224.895985677616 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(252,132,82)" stroke-opacity="1.0">
 
 
 
 <path d="M45.27221042399469 265.31701358372504 L45.27221042399469 265.31701358372504 L129.31044534278044 195.80054576204773 L205.2159478500762 38.53559504462828 L289.25418276886194 32.769284358835534 L370.5815068838201 55.355013917345445 L454.61974180260586 23.68181818181813 L535.9470659175568 106.8053151082882 L619.9853008363498 497.3181818181818 L704.0235357551355 351.59493972459325 L785.3508598700864 224.895985677616 L869.3890947888794 224.895985677616 L950.7164189038303 224.895985677616 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(84,112,198)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M45.27221042399469 329.12418